base= 30 cm

peralte efectivo = 50 - 8 = 42 cm

fc= 210 kgf / cm^2

n ( relacion modular ) = 9.2

dfrp = h + tf/2 = 50 + .12/2 = 50.6

In [ ]:
# Expresiones previas al diseño

phi_f=0.9 # Coeficiente de reduccion

def Area_acero(cant_aceros,diam):
  return cant_aceros*3.14*(diam*2.54)**2/4 #cm2

def Area_concreto(base,peralte_efectivo):
  return base*peralte_efectivo # cm2

def cuantia(Area_acero,Area_concreto):
  return Area_acero/Area_concreto # Adimensional

def w(cuantia,fy,fc): # Adimensional
  return cuantia*fy/fc # fy (kgf/cm2) , fc (kgf/cm2) , cuantia)

def Mn(base,peralte_efectivo,w,fc):
  return phi_f*(base*peralte_efectivo**2*fc*(1-.59*w)) # kgf*cm

def cuantia_balanceada(fc,fy): # fc (kgf/cm2)
  if fc/10<28:
    return (.85*.85*fc)/fy*(6117/(fy+6117))
  else:
    return (0.85-(0.05*(fc/10-28))/7)*.85*fc/fy*(6117/(fy+6117))

In [ ]:
round(Area_acero(3,5/8),2)

5.93

In [ ]:
round(Area_concreto(30,42),2)

1260

In [ ]:
round(cuantia(1.27,1260),4)

0.001

In [ ]:
round(w(0.0047,4200,210),4) # Adimensional

0.094

In [ ]:
round(Mn(30,42,0.094,210)*10**-5,2) # tnf-m

94.47

In [ ]:
round(cuantia_balanceada(210,4200),3)

0.021

In [ ]:
# Expresiones para el diseño con FRP
# Propiedades mecanicas
def Area_frp(numero_capas,espesor_frp,ancho_frp): # cm2
  return numero_capas*espesor_frp*ancho_frp

def cuantia_frp(Area_frp,Area_concreto): # Adimensional
  return Area_frp/Area_concreto

def k(cuantia_superior,relacion_modular): # Adimensional
  return -cuantia_superior*relacion_modular+((cuantia_superior*relacion_modular)**2 +2*cuantia_superior*relacion_modular)**.5

def Inercia_agrietada(base,k,peralte_efectivo,relacion_modular,area_acero_a_reforzar):
  return base*(k*peralte_efectivo)**3/3+relacion_modular*area_acero_a_reforzar*(peralte_efectivo-k*peralte_efectivo)**2
# Deformaciones

def ebi(Momento,d_frp,peralte_efectivo,k,Icr,Ec): # Momento (tnf-m, all in cm)
  return Momento*(d_frp-k*peralte_efectivo)/(Icr*Ec)

def efd(fc,n,Efrp,tf,efu):
  return .41*(fc/(n*Efrp*tf))**.5

def efe(ec,d_frp,c,ebi):
  return ec*(d_frp-c)/c-ebi

# Concreto y acero
def ec(efe,ebi,c,d_frp):
  return (efe+ebi)*c/(d_frp-c)

def es(efe,ebi,peralte_efectivo,d_frp,c):
  return (efe+ebi)*(peralte_efectivo-c)/(d_frp-c)

# Esfuerzos
def ffe(efe,E_frp):
  return efe*E_frp

def ecef(fc,Ec):
  return 1.7*fc/Ec

# Parametros
def beta(ecef,ec):
  return (4*ecef-ec)/(6*ecef-2*ec)

def alpha(ecef,ec,beta):
  return (3*ecef*ec-ec**2)/(3*beta*ecef**2)

# fuerzas para el Equilibrio
def cc(alpha,fc,beta,c,b):
  return alpha*fc*beta*c*b

def ts(acero_frp,fs):
  return acero_frp*fs

def tf(A_frp,ffe):
  return A_frp*ffe

def Mns(As_frp,fs,hef,beta,c):
  return As_frp*fs*(hef-beta*c/2)

def Mnf(A_frp,ffe,d_frp,beta,c):
  return A_frp*ffe*(d_frp-beta*c/2)

In [ ]:
round(Area_frp(1,0.12,30),2)

3.6

In [ ]:
round(cuantia_frp(3.6,30*42),3)

0.003

In [ ]:
round(k(0.001,2*10**6/(15100*210**.5)),2)

0.13

In [ ]:
round(Area_acero(3,5/8),2) # cm2

5.93

In [ ]:
round(Inercia_agrietada(30,0.13,42,9.2,5.93),2) # cm4

74469.34

In [ ]:
round(ebi(3.5,50.6,42,0.13,74469.34,15100*210**.5)*10**5,4)

0.001

In [ ]:
efd=round(efd(280,7.92,1650000,.12,0.015),4)
efd

0.0055

In [ ]:
round(.9*.95*0.017,3) # Limite para el valor anterior

0.015

In [ ]:
round(efe(0.003,50.6,10,0.001),3)

0.011

In [ ]:
if round(efe(0.003,50.6,10,0.001),3) > efd:
  print('ec<ecu')
else:
  print('ec=ecu')

ec<ecu


In [ ]:
round(ec(0.0055,0.001,10,50.6),4)

0.0016

In [ ]:
round(es(0.0055,0.001,42,50.6,10),3)

0.005

In [ ]:
round(ffe(0.0055,1650000),2)

9075.0

In [ ]:
if round(ffe(0.0055,1650000),2)>31000*.95:
  print ('frp falla')
else:
  print ('frp no falla aun')

frp no falla aun


In [ ]:
ffe(0.0055,1650000),31000*.95

(9075.0, 29450.0)

In [ ]:
round(ecef(210,15100*210**.5),4)

0.0016

In [ ]:
round(beta(round(ecef(210,15100*210**.5),4),0.0016),2)

0.75

In [ ]:
round(alpha(0.0016,0.0016,0.79),2)

0.84

In [ ]:
round(cc(.84,280,.75,10,30),2)

52920.0

In [ ]:
round (ts(Area_acero(3,5/8),4200),2)

24926.87

In [ ]:
round(tf(3.6,9075),2)

32670.0

In [ ]:
Mns(Area_acero(3,5/8),4200,42,.75,10)*10**-5

9.534526041796875

In [ ]:
Mnf(3.6,9075,50.6,.75,10)*10**-5

15.305895000000001